In [17]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
df = pd.read_csv('D:/College/sem 7/final year project/gith/AniSearch/anime.csv')
df = df.reset_index()
df.head()

,index,Anime-PlanetID,Name,Alternative Name,Rating Score,Number Votes,Tags,Content Warning,Type,Episodes,Finished,Duration,StartYear,EndYear,Season,Studios,Synopsis,Url
0,0,10,The Prince of Tennis,Tennis no Ouji-sama,4.037,10889,"Comedy, Drama, Shounen, Sports, Tennis, Based ...",Unknown,TV,178,True,Unknown,2001,2005,Fall 2001,"Production I.G, Trans Arts","Meet Ryoma Echizen, the cocky prince of tennis...",https://www.anime-planet.com/anime/the-prince-...
1,1,100,Neon Genesis Evangelion,Shinseiki Evangelion,4.248,54463,"Drama, Mecha, Sci Fi, Conspiracy, Kaijuu, Lone...","Emotional Abuse, Explicit Violence, Mature The...",TV,26,True,Unknown,1995,1996,Fall 1995,"GAINAX, Tatsunoko Production","In the future, a devastating event known as Se...",https://www.anime-planet.com/anime/neon-genesi...
2,2,1000,Full Metal Panic! The Second Raid,Unknown,4.35,23948,"Action, Comedy, Mecha, Sci Fi, Shounen, Milita...","Explicit Violence, Nudity",TV,13,True,Unknown,2005,2005,Summer 2005,Kyoto Animation,Half a year has passed since Sousuke Sagara to...,https://www.anime-planet.com/anime/full-metal-...
3,3,10000,22/7: Shampoo no Nioi ga Shita,Unknown,2.8,131,"Idols, School Life, CG Animation",Unknown,Music,1,True,5,2018,2018,Unknown,Unknown,The idol group 22/7 perform the song Shampoo n...,https://www.anime-planet.com/anime/22-7-shampo...
4,4,10001,Oshiri Tantei: Puputto Fumutto Kaiketsu Dance,Unknown,1.271,21,Family Friendly,Unknown,Web,1,True,2,2017,2017,Unknown,Toei Animation,No synopsis yet - check back soon!,https://www.anime-planet.com/anime/oshiri-tant...


In [19]:
df.columns

Index(['index', 'Anime-PlanetID', 'Name', 'Alternative Name', 'Rating Score',
       'Number Votes', 'Tags', 'Content Warning', 'Type', 'Episodes',
       'Finished', 'Duration', 'StartYear', 'EndYear', 'Season', 'Studios',
       'Synopsis', 'Url'],
      dtype='object')

In [20]:
features = ['Rating Score','Number Votes','Studios','Synopsis', 'Tags', 'Episodes']

In [21]:
def combined_features(row):
    return str(row["Rating Score"])+" "+ str(row["Number Votes"])+" "+ str(row["Studios"])+" "+ str(row["Synopsis"])+" "+ str(row["Tags"])+" "+ str(row["Episodes"])+" "

def get_title_from_index(index):
    return df[df["index"] == index]["Name"].values[0]
def get_index_from_title(title):
    return df[df["Name"] == title]["index"].values[0]

In [22]:
df["combined_feature"]=df.apply(combined_features,axis=1)
df["combined_feature"].head()

0    4.037 10889 Production I.G, Trans Arts Meet Ry...
1    4.248 54463 GAINAX, Tatsunoko Production In th...
2    4.35 23948 Kyoto Animation Half a year has pas...
3    2.8 131 Unknown The idol group 22/7 perform th...
4    1.271 21 Toei Animation No synopsis yet - chec...
Name: combined_feature, dtype: object

In [23]:
cv = CountVectorizer()
count_matrix=cv.fit_transform(df["combined_feature"])

In [24]:
anime_liked = "Attack on titan"


In [25]:
cosine_sim = cosine_similarity(count_matrix)
from difflib import get_close_matches


def get_index_from_title(title):
    title = title.strip().lower()
    df["Name"] = df["Name"].str.strip().str.lower()
    matches = get_close_matches(title, df["Name"], n=1, cutoff=0.8)  
    if matches:
        title = matches[0]  
        return df[df["Name"] == title]["index"].values[0]
    else:
        raise ValueError(f"No close matches found for '{title}'.")

try:
    liked_movie_index = cosine_sim[get_index_from_title(anime_liked)]
    similar_anime = list(enumerate(liked_movie_index))
    similar_anime.sort(key=lambda row: row[1], reverse=True)
    for i in range(11):
        print(get_title_from_index(similar_anime[i][0]))
except Exception as e:
    print(f"Error: {e}")


attack on titan
attack on titan: since that day
triton of the sea
utopa
attack on titan 3rd season
mazinger z vs dark general
record of lodoss war
slayers great
pumpkin scissors
genocidal organ
attack on titan 3rd season: part ii


#### Recommendation using Cosine Similarity

In [26]:
from difflib import get_close_matches
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

# Load and clean data
df['Tags'] = df['Tags'].fillna('')
df['Studios'] = df['Studios'].fillna('')
df['Type'] = df['Type'].fillna('Unknown')
df['Synopsis'] = df['Synopsis'].fillna('')
df['Rating Score'] = pd.to_numeric(df['Rating Score'], errors='coerce')
df['Number Votes'] = pd.to_numeric(df['Number Votes'], errors='coerce').fillna(0)

# Preprocessing
scaler = StandardScaler()
scaled_votes = scaler.fit_transform(df[['Number Votes']])

tag_vectorizer = TfidfVectorizer(stop_words='english', max_features=600)
tag_matrix = tag_vectorizer.fit_transform(df['Tags']).toarray()

synopsis_vectorizer = TfidfVectorizer(stop_words='english', max_features=1500)
synopsis_matrix = synopsis_vectorizer.fit_transform(df['Synopsis']).toarray()

encoder = OneHotEncoder()
studios_encoded = encoder.fit_transform(df[['Studios']]).toarray()
type_encoded = encoder.fit_transform(df[['Type']]).toarray()



# Recommendation helper functions
def get_index_from_title(title):
    title = title.strip().lower()
    df["Name"] = df["Name"].str.strip().str.lower()
    matches = get_close_matches(title, df["Name"], n=1, cutoff=0.8)
    if matches:
        title = matches[0]
        return df[df["Name"] == title]["index"].values[0]
    else:
        raise ValueError(f"No close matches found for '{title}'.")

def get_title_from_index(index):
    return df[df.index == index]["Name"].values[0]

# Example recommendation (requires cosine_sim to be defined elsewhere)
try:
    # anime_liked = "Naruto"  # Example input
    liked_movie_index = cosine_sim[get_index_from_title(anime_liked)]
    similar_anime = list(enumerate(liked_movie_index))
    similar_anime.sort(key=lambda row: row[1], reverse=True)

    print(f"\nRecommendations for {anime_liked} (Cosine Similarity):")
    for i in range(11):
        print(get_title_from_index(similar_anime[i][0]))
except Exception as e:
    print(f"Error: {e}")


features_combined = np.hstack([scaled_votes, tag_matrix, synopsis_matrix, studios_encoded, type_encoded])
print("Feature matrix shape:", features_combined.shape)


Recommendations for Attack on titan (Cosine Similarity):
attack on titan
attack on titan: since that day
triton of the sea
utopa
attack on titan 3rd season
mazinger z vs dark general
record of lodoss war
slayers great
pumpkin scissors
genocidal organ
attack on titan 3rd season: part ii
Feature matrix shape: (16621, 3153)


##### KNN using Manhattan (without normalization)

In [28]:
from scipy.spatial.distance import cdist
from difflib import get_close_matches

def get_index_from_title(title):
    title = title.strip().lower()
    df["Name"] = df["Name"].str.strip().str.lower()
    matches = get_close_matches(title, df["Name"], n=1, cutoff=0.8)  
    if matches:
        title = matches[0]  
        return df[df["Name"] == title]["index"].values[0]
    else:
        raise ValueError(f"No close matches found for '{title}'.")

try:
    liked_movie_index = get_index_from_title(anime_liked)
    similarity_scores = cdist([features_combined[liked_movie_index]], features_combined, metric='cityblock').flatten()
    similar_anime = list(enumerate(similarity_scores))
    similar_anime.sort(key=lambda row: row[1])  # Lower Manhattan distance means higher similarity

    print(f"Anime similar to '{anime_liked}' based on Manhattan Distance:\n")
    for i in range(11):
        anime_name = get_title_from_index(similar_anime[i][0])
        distance = similar_anime[i][1]
        similarity_score = 1 / (1 + distance)  # Convert distance to similarity (closer = higher similarity)
        print(f"{anime_name} - \t\tDistance: {distance:.4f}, \t\tSimilarity: {similarity_score:.4f}")

except Exception as e:
    print(f"Error: {e}")


Anime similar to 'Attack on titan' based on Manhattan Distance:

attack on titan - 		Distance: 0.0000, 		Similarity: 1.0000
naruto - 		Distance: 19.3631, 		Similarity: 0.0491
fullmetal alchemist: brotherhood - 		Distance: 21.2993, 		Similarity: 0.0448
naruto shippuden - 		Distance: 21.8002, 		Similarity: 0.0439
fullmetal alchemist - 		Distance: 21.9757, 		Similarity: 0.0435
attack on titan: since that day - 		Distance: 22.0638, 		Similarity: 0.0434
attack on titan 2nd season - 		Distance: 22.6179, 		Similarity: 0.0423
sword art online - 		Distance: 23.7826, 		Similarity: 0.0404
bleach - 		Distance: 24.1498, 		Similarity: 0.0398
death note - 		Distance: 24.5408, 		Similarity: 0.0392
one-punch man - 		Distance: 24.8193, 		Similarity: 0.0387


##### KNN using Euclidean (without normalization)

In [29]:
from scipy.spatial.distance import cdist

try:
    liked_movie_index = get_index_from_title(anime_liked)
    similarity_scores = cdist([features_combined[liked_movie_index]], features_combined, metric='euclidean').flatten()
    # Find Min & Max for Normalization
    min_distance = np.min(similarity_scores)
    max_distance = np.max(similarity_scores)

    similar_anime = list(enumerate(similarity_scores))
    similar_anime.sort(key=lambda row: row[1])  # Lower Euclidean distance means higher similarity

    print(f"Anime similar to '{anime_liked}' based on Euclidean Distance:\n")
    for i in range(11):
        anime_name = get_title_from_index(similar_anime[i][0])
        distance = similar_anime[i][1]
        similarity_score = 1 / (1 + distance)
        print(f"{anime_name} - \t\tDistance: {distance:.4f}, \t\tSimilarity: {similarity_score:.4f}")

except Exception as e:
    print(f"Error: {e}")


Anime similar to 'Attack on titan' based on Euclidean Distance:

attack on titan - 		Distance: 0.0000, 		Similarity: 1.0000
naruto - 		Distance: 2.6293, 		Similarity: 0.2755
sword art online - 		Distance: 3.6853, 		Similarity: 0.2134
fullmetal alchemist: brotherhood - 		Distance: 4.5835, 		Similarity: 0.1791
spirited away - 		Distance: 5.1998, 		Similarity: 0.1613
naruto shippuden - 		Distance: 5.2078, 		Similarity: 0.1611
death note - 		Distance: 5.4614, 		Similarity: 0.1548
bleach - 		Distance: 5.6542, 		Similarity: 0.1503
fullmetal alchemist - 		Distance: 5.6951, 		Similarity: 0.1494
code geass: lelouch of the rebellion - 		Distance: 6.8355, 		Similarity: 0.1276
one-punch man - 		Distance: 6.9046, 		Similarity: 0.1265


In [30]:
from scipy.spatial.distance import cdist
from difflib import get_close_matches

def get_index_from_title(title):
    title = title.strip().lower()
    df["Name"] = df["Name"].str.strip().str.lower()
    matches = get_close_matches(title, df["Name"], n=1, cutoff=0.8)  
    if matches:
        title = matches[0]  
        return df[df["Name"] == title]["index"].values[0]
    else:
        raise ValueError(f"No close matches found for '{title}'.")

# Code for Cityblock (Manhattan) Distance
def find_similar_anime_cityblock(anime_liked):
    try:
        liked_movie_index = get_index_from_title(anime_liked)
        similarity_scores = cdist([features_combined[liked_movie_index]], features_combined, metric='cityblock').flatten()
        similar_anime = list(enumerate(similarity_scores))
        similar_anime.sort(key=lambda row: row[1])  # Lower distance means higher similarity

        print(f"Anime similar to '{anime_liked}' based on Cityblock Distance:\n")
        for i in range(11):
            anime_name = get_title_from_index(similar_anime[i][0])
            distance = similar_anime[i][1]
            similarity_score = 1 / (1 + distance)  # Convert distance to similarity (closer = higher similarity)
            print(f"{anime_name} - \t\tDistance: {distance:.4f}, \t\tSimilarity: {similarity_score:.4f}")

    except Exception as e:
        print(f"Error: {e}")

# Code for Chebyshev Distance
def find_similar_anime_chebyshev(anime_liked):
    try:
        liked_movie_index = get_index_from_title(anime_liked)
        similarity_scores = cdist([features_combined[liked_movie_index]], features_combined, metric='chebyshev').flatten()
        similar_anime = list(enumerate(similarity_scores))
        similar_anime.sort(key=lambda row: row[1])  # Lower distance means higher similarity

        print(f"Anime similar to '{anime_liked}' based on Chebyshev Distance:\n")
        for i in range(11):
            anime_name = get_title_from_index(similar_anime[i][0])
            distance = similar_anime[i][1]
            similarity_score = 1 / (1 + distance)  # Convert distance to similarity (closer = higher similarity)
            print(f"{anime_name} - \t\tDistance: {distance:.4f}, \t\tSimilarity: {similarity_score:.4f}")

    except Exception as e:
        print(f"Error: {e}")

# Example usage:
# anime_liked = "Your Favorite Anime Title Here"
# find_similar_anime_cityblock(anime_liked)
find_similar_anime_chebyshev(anime_liked)


Anime similar to 'Attack on titan' based on Chebyshev Distance:

attack on titan - 		Distance: 0.0000, 		Similarity: 1.0000
naruto - 		Distance: 1.1280, 		Similarity: 0.4699
sword art online - 		Distance: 2.8220, 		Similarity: 0.2616
fullmetal alchemist: brotherhood - 		Distance: 3.9955, 		Similarity: 0.2002
spirited away - 		Distance: 4.3904, 		Similarity: 0.1855
naruto shippuden - 		Distance: 4.6444, 		Similarity: 0.1772
death note - 		Distance: 4.9108, 		Similarity: 0.1692
bleach - 		Distance: 5.1143, 		Similarity: 0.1635
fullmetal alchemist - 		Distance: 5.2252, 		Similarity: 0.1606
code geass: lelouch of the rebellion - 		Distance: 6.4156, 		Similarity: 0.1349
one-punch man - 		Distance: 6.5070, 		Similarity: 0.1332


In [31]:
from scipy.spatial.distance import cdist

def find_similar_anime_correlation(anime_liked):
    try:
        liked_movie_index = get_index_from_title(anime_liked)
        similarity_scores = cdist([features_combined[liked_movie_index]], features_combined, metric='correlation').flatten()
        similar_anime = sorted(enumerate(similarity_scores), key=lambda x: x[1])

        print(f"Anime similar to '{anime_liked}' based on Correlation Distance:\n")
        for i in range(11):
            anime_name = get_title_from_index(similar_anime[i][0])
            distance = similar_anime[i][1]
            similarity_score = 1 - distance  # Closer correlation distance = higher similarity
            print(f"{anime_name} - Distance: {distance:.4f}, Similarity: {similarity_score:.4f}")

    except Exception as e:
        print(f"Error: {e}")
find_similar_anime_correlation(anime_liked)

Anime similar to 'Attack on titan' based on Correlation Distance:

attack on titan - Distance: 0.0000, Similarity: 1.0000
death note - Distance: 0.0057, Similarity: 0.9943
naruto - Distance: 0.0071, Similarity: 0.9929
fullmetal alchemist: brotherhood - Distance: 0.0078, Similarity: 0.9922
sword art online - Distance: 0.0079, Similarity: 0.9921
fullmetal alchemist - Distance: 0.0088, Similarity: 0.9912
naruto shippuden - Distance: 0.0090, Similarity: 0.9910
bleach - Distance: 0.0098, Similarity: 0.9902
one-punch man - Distance: 0.0103, Similarity: 0.9897
code geass: lelouch of the rebellion - Distance: 0.0106, Similarity: 0.9894
elfen lied - Distance: 0.0114, Similarity: 0.9886


### Minkowski

In [36]:
from scipy.spatial.distance import cdist
from difflib import get_close_matches
import numpy as np

# Function to get index of an anime from its title
def get_index_from_title(title):
    title = title.strip().lower()
    df["Name"] = df["Name"].str.strip().str.lower()
    matches = get_close_matches(title, df["Name"], n=1, cutoff=0.8)
    if matches:
        title = matches[0]
        return df[df["Name"] == title]["index"].values[0]
    else:
        raise ValueError(f"No close matches found for '{title}'.")

try:
    liked_movie_index = get_index_from_title(anime_liked)
    
    # Compute Minkowski Distance for all anime (p=3)
    similarity_scores = cdist([features_combined[liked_movie_index]], features_combined, metric='minkowski', p=3).flatten()
    
    similar_anime = list(enumerate(similarity_scores))
    similar_anime.sort(key=lambda row: row[1])  # Lower distance means higher similarity
    
    print(f"Anime similar to '{anime_liked}' based on Minkowski Distance (p=3):\n")
    for i in range(11):
        anime_name = get_title_from_index(similar_anime[i][0])
        distance = similar_anime[i][1]
        similarity_score = 1 / (1 + distance)  # Convert distance to similarity
        print(f"{anime_name} - \t\tDistance: {distance:.4f}, \t\tSimilarity: {similarity_score:.4f}")

except Exception as e:
    print(f"Error: {e}")




Anime similar to 'Attack on titan' based on Minkowski Distance (p=3):

attack on titan - 		Distance: 0.0000, 		Similarity: 1.0000
naruto - 		Distance: 1.6459, 		Similarity: 0.3779
sword art online - 		Distance: 2.9335, 		Similarity: 0.2542
fullmetal alchemist: brotherhood - 		Distance: 4.0507, 		Similarity: 0.1980
spirited away - 		Distance: 4.4738, 		Similarity: 0.1827
naruto shippuden - 		Distance: 4.6935, 		Similarity: 0.1756
death note - 		Distance: 4.9503, 		Similarity: 0.1681
bleach - 		Distance: 5.1529, 		Similarity: 0.1625
fullmetal alchemist - 		Distance: 5.2586, 		Similarity: 0.1598
code geass: lelouch of the rebellion - 		Distance: 6.4381, 		Similarity: 0.1344
one-punch man - 		Distance: 6.5285, 		Similarity: 0.1328
